In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from pandas_profiling import ProfileReport
import pickle
import os
import datetime
import calendar
from scipy.stats import zscore
import missingno as msno

In [121]:
nday=10 # Number of days to roll

In [125]:
df_weather = pd.read_csv('IC_Weather_Dat.csv')
df_weather['date_time'] = pd.to_datetime(df_weather['date_time'])
df_weather.drop(df_weather[df_weather['date_time'].dt.year.isin([2023, 2014, 2015])].index, axis=0, inplace=True)
df_precip = df_weather[['date_time', 'wetstate_avg', 'precip_tb', 'precip_wpg', 'precip_7day']]
df_precip['precip_nday'] = df_precip['precip_tb'].shift(-nday).rolling(nday).sum()

df_temp = df_weather[['date_time', 'airt_avg']]

C:\Users\Brayden\AppData\Local\Temp\ipykernel_149016\3226655641.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [3]:
df_ic_routes, _, _ = pickle.load(open('C:/Users/Brayden/Desktop/LocScrapes/Indian Creek (Full).pkl', 'rb'))
df_ic_routes.drop_duplicates(subset='URL', inplace=True)
df_ic_routes['Base Location'] = df_ic_routes['Base Location'].apply(lambda x: x.strip())

In [4]:
df_super_tick_list = pd.concat(df_ic_routes['Route Ticks'].to_list())

In [5]:
(df_precip.isnull().sum()/df_precip.shape[0])*100

date_time        0.000000
wetstate_avg    49.252557
precip_tb        0.000000
precip_wpg      14.358773
precip_7day      0.000000
dtype: float64

In [6]:
px.line(data_frame=df_precip, x='date_time', y=['precip_tb'])

In [8]:
yearly_precip = df_precip.groupby(df_precip['date_time'].dt.year)['precip_tb'].sum()
# px.bar(yearly_precip)

In [9]:
df_monthly_precip = df_precip.groupby([df_precip['date_time'].dt.year, df_precip['date_time'].dt.month])['precip_tb'].sum().unstack(level=1)
monthly_precip_mean = df_monthly_precip.mean(axis=0)
px.bar(monthly_precip_mean, title='Average Monthly Sum Rainfall Over 2016-2022 (in)')
# This is sum of rainfall by month, then averaged over the years.
# March April May is very rainy. October less so but November is actually very good.

In [10]:
df_precip_value_counts = df_precip.groupby(df_precip['date_time'].dt.month)['precip_tb'].value_counts().unstack()
df_precip_noprecip_rate = df_precip_value_counts[0.00] / df_precip_value_counts.sum(axis=1)
px.bar(df_precip_noprecip_rate, title='Percentage of Days without Rain')

In [11]:
tick_sum_by_month = df_super_tick_list.groupby(df_super_tick_list['Date'].dt.month)['Username'].count()
px.bar(tick_sum_by_month, title='Tick Sum by Month')

In [119]:
temp_bins=[0, 30, 50, 65, 80, 100]

df_temp_groupings = df_temp.groupby(df_temp['date_time'].dt.month)['airt_avg'].value_counts(bins=temp_bins, sort=False).unstack().values / pd.concat([df_temp.groupby(df_temp['date_time'].dt.month)['airt_avg'].count()]*(len(temp_bins)-1), axis=1).values
df_temp_groupings = pd.DataFrame(df_temp_groupings, columns=['Cold', 'Chilly', 'Prime', 'Warm', 'Hot'], index=[1,2,3,4,5,6,7,8,9,10,11,12])

temp_color_map = {'Cold':'rgb(148, 103, 189)', 'Chilly':'rgb(31, 119, 180)', 'Prime':'rgb(44, 160, 44)', 'Warm':'rgb(255, 127, 14)', 'Hot':'rgb(214, 39, 40)'}
fig = px.bar(df_temp_groupings, color_discrete_map=temp_color_map)
fig

In [131]:
df_temp

,date_time,airt_avg
602,2016-01-01 23:59:59,11.6
603,2016-01-02 23:59:59,16.4
604,2016-01-03 23:59:59,18.5
605,2016-01-04 23:59:59,21.7
606,2016-01-05 23:59:59,27.7
...,...,...
3139,2022-12-26 23:59:59,33.1
3140,2022-12-27 23:59:59,41.3
3141,2022-12-28 23:59:59,35.1
3142,2022-12-29 23:59:59,28.1


In [162]:
# Rolling 10day score for quality of temperature that day. Prime is 0 points, chilly or warm is -1 points, hot or cold is -2 points
df_temp['points'] = pd.cut(df_temp['airt_avg'], bins=temp_bins, labels=[2, 1, 0, 1, 2], ordered=False)
df_temp['nday_sum'] = df_temp['points'].rolling(nday).sum().shift(-nday)
mean_temp_points = df_temp.groupby(df_temp['date_time'].dt.dayofyear)['nday_sum'].mean()
px.bar(mean_temp_points, title='Forward Rolling nday Temperature Point Value')

C:\Users\Brayden\AppData\Local\Temp\ipykernel_149016\2313983094.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Brayden\AppData\Local\Temp\ipykernel_149016\2313983094.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [164]:
mean_precip_nday = df_precip.groupby(df_precip['date_time'].dt.dayofyear)['precip_nday'].mean()
px.line(mean_precip_nday, title='Forward Rolling nday Precip Mean')

In [161]:
# Want a forward rolling nday days without precip
df_precip[df_precip['precip_nday']==0]['date_time'].dt.month.value_counts().sort_index()

1      59
2      46
3      29
4      38
5      96
6     116
7      30
8      94
9      61
10     97
11     93
12     51
Name: date_time, dtype: int64

In [170]:
df_super_tick_list

,Username,User Link,Date,Pitches Ticked,Style,Lead Style,Comment
0,billy w,/user/112002192,2022-11-25,1.0,Lead,Onsight,
1,Finn O'Hea,/user/200846100,2022-11-22,1.0,Lead,Onsight,The finest desert tower of them all
2,Cole Bellomo,/user/200709105,2022-11-13,1.0,Lead,Flash,
3,Véro Rochat,/user/201488919,2022-11-13,1.0,Lead,Onsight,Joli pas bloc au départ : devers avec les main...
4,mia moore,/user/201478310,2022-11-07,1.0,,,barrett
...,...,...,...,...,...,...,...
0,Angelo R,/user/111170121,2021-02-28,1.0,Follow,,"Super good climbing! Good job, Matt!"
1,Matthew Tangeman,/user/110456934,2021-02-27,1.0,Lead,Redpoint,first ascent
0,Brandon Adams,/user/106918913,2021-04-01,1.0,,,FA
0,Josh Bender,/user/200079461,2021-11-21,1.0,Lead,Onsight,FA with Gus and Thad


In [179]:
# Want a forward rolling nday tick values
df_super_tick_list_year_limited = df_super_tick_list[df_super_tick_list['Date'].dt.year.isin([2016, 2018, 2019, 2020, 2021, 2022])]
df_tick_by_date = df_super_tick_list_year_limited.groupby('Date')['Username'].count().sort_index()
df_tick_by_date

Date
2016-01-01    1
2016-01-03    2
2016-01-04    3
2016-01-08    2
2016-01-12    2
             ..
2022-12-22    4
2022-12-23    2
2022-12-24    8
2022-12-25    7
2022-12-26    2
Name: Username, Length: 1817, dtype: int64